In [19]:
import pytest
from memories.auto_album import AutoAlbum
from memories.connection import get_connection
import pandas.io.sql as sqlio
import numpy as np
import warnings
warnings.filterwarnings('ignore')


In [20]:
class TestAutoAlbum:
    def __init__(self):    
        self.connections = []
        
    def test_AutoAblum_creation(self):
        self.connections.append(get_connection())
        # Initial check of albums count
        data = sqlio.read_sql_query("SELECT * FROM auto_album", self.connections[-1])
        auto_album_count0 = len(data.head())
        data = sqlio.read_sql_query("SELECT * FROM normal_album", self.connections[-1])
        album_count0 = len(data.head())
        # Creating album 
        new_album = AutoAlbum('remy',
                      self.connections[-1],
                      'Test',
                     )
        # Checking album counts have been incremented
        auto_data = sqlio.read_sql_query("SELECT * FROM auto_album", self.connections[-1])
        auto_album_count1 = len(auto_data.head())
        data = sqlio.read_sql_query("SELECT * FROM normal_album", self.connections[-1])
        album_count1 = len(data.head())
        np.testing.assert_equal(auto_album_count1, auto_album_count0+1)
        np.testing.assert_equal(album_count1, album_count0+1)
        # Checking creation time
        test_data = sqlio.read_sql_query("SELECT * FROM auto_album WHERE album_name = 'Test'", self.connections[-1])
        np.testing.assert_equal(test_data['epoch_created'][0], test_data['epoch_last_modified'][0])
        # Trying to create the same album againn
        new_album = AutoAlbum('remy',
                      self.connections[-1],
                      'Test',
                     )

        # Check album counts were not increased
        data = sqlio.read_sql_query("SELECT * FROM auto_album", self.connections[-1])
        auto_album_count2 = len(data.head())
        data = sqlio.read_sql_query("SELECT * FROM normal_album", self.connections[-1])
        album_count2 = len(data.head())
        np.testing.assert_equal(auto_album_count1, auto_album_count2)
        np.testing.assert_equal(album_count1, album_count2)
        # Assert right album id was found
        data = sqlio.read_sql_query("SELECT album_id FROM auto_album WHERE album_name = 'Test'", self.connections[-1])
        np.testing.assert_equal(new_album.album_id, data.head()['album_id'][0])

        new_album = AutoAlbum('aline',
                      self.connections[-1],
                      'Test',
                     )
        data = sqlio.read_sql_query("SELECT * FROM normal_album", self.connections[-1])
        album_count3 = len(data.head())
        np.testing.assert_equal(album_count3, album_count2+1)

    def test_album_update(self):
        
        self.connections.append(get_connection())
        data = sqlio.read_sql_query("SELECT * FROM public.many_item_has_many_normal_album",
                                    self.connections[-1])
        count0 = len(data.head())

        new_album = AutoAlbum('remy',
                      self.connections[-1],
                      'Test',
                     )

        new_album.update_album(start=str(1673911228),
                     stop=str(1674775228),
                     people=None,
                     in_album=None,
                     commit=False)
        data = sqlio.read_sql_query("SELECT item_count FROM public.normal_album WHERE name = 'Test' AND id_user = 1;",
                                    self.connections[-1]).values[0][0]
        assert data
        np.testing.assert_equal(data, 15)

        new_album.update_album(start=str(1673911228),
                     stop=str(1674775228),
                     people=None,
                     in_album=None,
                     commit=False)

        
    def test_album_share(self):
        
        self.connections.append(get_connection())
        data = sqlio.read_sql_query("SELECT * FROM auto_album", self.connections[-1])
        auto_album_count0 = len(data.head())
        data = sqlio.read_sql_query("SELECT * FROM normal_album", self.connections[-1])
        album_count0 = len(data.head())
        new_album = AutoAlbum('remy',
                      self.connections[-1],
                      'Test_share',
                      shared=True,
                     )
        
        
        auto_data = sqlio.read_sql_query("SELECT * FROM auto_album", self.connections[-1])
        auto_album_count1 = len(auto_data.head())
        data = sqlio.read_sql_query("SELECT * FROM normal_album", self.connections[-1])
        album_count1 = len(data.head())
        np.testing.assert_equal(auto_album_count1, auto_album_count0+1)
        np.testing.assert_equal(album_count1, album_count0+1)

        data = sqlio.read_sql_query("SELECT * FROM normal_album WHERE name = 'Test_share'", self.connections[-1])
        auto_data = sqlio.read_sql_query("SELECT * FROM auto_album WHERE album_name = 'Test_share'", self.connections[-1])
        np.testing.assert_equal(data['shared'][0], True)
        np.testing.assert_equal(auto_data['shared'][0], True)

        new_album.update_album(start=str(1673911228),
                               stop=str(1674775228),
                               limit=12,
                               in_album=4,)
        data = sqlio.read_sql_query("SELECT item_count FROM public.normal_album WHERE name = 'Test_share' AND id_user = 1;",
                                    self.connections[-1]).values[0][0]
        assert data
        np.testing.assert_equal(data, 12)

    def test_close_connections(self):
        for conn in self.connections:
            conn.rollback()
            conn.close

In [21]:
test = TestAutoAlbum()
test.test_AutoAblum_creation()

Connection to the PostgreSQL established successfully.
1, 'Test', 'TEST', NULL, 0, false, 1737578022, 0, 0, 0
Album Test already exists. Not creating new album.
2, 'Test', 'TEST', NULL, 0, false, 1737578022, 0, 0, 0


In [22]:
test.test_album_update()

Connection to the PostgreSQL established successfully.
1, 'Test', 'TEST', NULL, 0, false, 1737578023, 0, 0, 0


In [23]:
test_share = TestAutoAlbum()
test_share.test_album_share()

Connection to the PostgreSQL established successfully.
1, 'Test_share', 'TEST_SHARE', 'TEST_SHARE_remy_pass', 0, true, 1737578023, 0, 0, 0


In [41]:
conn = get_connection()
data = sqlio.read_sql_query("SELECT * FROM auto_album", test.connections[0])
# Now data is a pandas dataframe having the results of above query.
data.head()

Connection to the PostgreSQL established successfully.


,album_id,user_id,album_name,epoch_created,epoch_last_modified,shared
0,26,1,Sedona aujourd`hui,1737588756,1737588756,True
1,21,8,Sedona et Moi,1737588555,1737588973,False
2,19,2,Sedona et Moi,1737578040,1737588973,False
3,22,5,Sedona et Moi,1737588634,1737588974,False
4,20,1,Sedona et Moi,1737583452,1737588974,False


In [43]:

data = sqlio.read_sql_query("SELECT * FROM normal_album", test_share.connections[-1])
# Now data is a pandas dataframe having the results of above query.
data.head(20)

,id,id_user,name,type,shared,create_time,cover,sort_by,sort_direction,normalized_name,version,passphrase_share,item_count,cant_migrate_condition,condition,status,start_time,end_time,name_for_sort
0,4,1,Sedona,0,True,1722110300,16372,0,0,SEDONA,129379,SNSRSdd4K,4402.0,None,None,0,1.673757e+09,1.737580e+09,SEDONA
1,10,2,Love and Travel,0,False,1737189318,18828,0,0,LOVE AND TRAVEL,114945,None,772.0,None,None,0,1.335888e+09,1.737234e+09,LOVE AND TRAVEL
2,18,1,Test_share,0,True,1737578023,3467,0,0,TEST_SHARE,129410,TEST_SHARE_remy_pass,12.0,None,None,0,1.674061e+09,1.674745e+09,
3,26,1,Sedona aujourd`hui,0,True,1737588756,0,0,0,SEDONA AUJOURD`HUI,130211,SEDONA AUJOURD`HUI_remy_pass,NaN,None,None,0,NaN,NaN,
4,21,8,Sedona et Moi,0,False,1737588555,5281,0,0,SEDONA ET MOI,130214,None,2.0,None,None,0,1.705766e+09,1.705767e+09,
5,19,2,Sedona et Moi,0,False,1737578040,5870,0,0,SEDONA ET MOI,130226,None,10.0,None,None,0,1.674319e+09,1.728225e+09,
6,22,5,Sedona et Moi,0,False,1737588634,2126,0,0,SEDONA ET MOI,130240,None,12.0,None,None,0,1.678019e+09,1.717960e+09,
7,20,1,Sedona et Moi,0,False,1737583452,10502,0,0,SEDONA ET MOI,130254,None,10.0,None,None,0,1.685980e+09,1.728225e+09,
8,23,9,Sedona et Moi,0,False,1737588635,2352,0,0,SEDONA ET MOI,130263,None,8.0,None,None,0,1.694874e+09,1.717669e+09,
9,24,7,Sedona et Moi,0,False,1737588635,5796,0,0,SEDONA ET MOI,130276,None,13.0,None,None,0,1.685299e+09,1.728067e+09,


In [44]:
data = sqlio.read_sql_query("SELECT * FROM public.share", test_share.connections[-1])
print(len(data))
# Now data is a pandas dataframe having the results of above query.
data.head(15)

5


,passphrase,privacy_type,modified_time,id_user,expired_time,hashed_password,enable,type
0,SNSRSdd4K,0,1735811147,1,0,,True,0
1,LbrIc3r9H,0,1723031220,0,0,,False,1
2,slaJo9jFF,0,1723031222,0,0,,False,1
3,TEST_SHARE_remy_pass,0,1737578023,1,0,,True,0
4,SEDONA AUJOURD`HUI_remy_pass,0,1737588756,1,0,,True,0


In [40]:
data = sqlio.read_sql_query("SELECT * FROM public.share_permission ", conn)
print(len(data))
# Now data is a pandas dataframe having the results of above query.
data.head(25)

18


,id,id_user,permission,target_type,target_id,passphrase_share
0,6,0,31,1,0,slaJo9jFF
1,7,0,31,1,0,LbrIc3r9H
2,4,1,31,1,1,SNSRSdd4K
3,15,1,7,1,8,SNSRSdd4K
4,5,1,7,1,2,SNSRSdd4K
5,14,1,7,1,7,SNSRSdd4K
6,16,1,1,1,9,SNSRSdd4K
7,12,1,7,1,5,SNSRSdd4K
8,13,1,7,1,6,SNSRSdd4K
9,11,1,7,1,4,SNSRSdd4K


In [16]:
conn.rollback()

In [17]:
conn.close()

In [18]:
test.test_close_connections()
test_share.test_close_connections()

InterfaceError: connection already closed

In [47]:
import datetime as dt
album = AutoAlbum('remy',
                          conn,
                          album_name="Sedona aujourd`hui",
                          shared=True,
                          commit=True,
                          )

album.update_album(start=dt.datetime.now().timestamp() - 24 * 3600,
                           stop=dt.datetime.now().timestamp(),
                           limit = 10 + int(np.random.rand(1) * 5),
                           in_album = 4,
                           commit = True)

Album Sedona aujourd`hui already exists. Not creating new album.
Album creation failed with error: index 0 is out of bounds for axis 0 with size 0
